In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json

new_boundary = gpd.read_file("preprocess/data/new_boundary/new_boundary.shp")
geojson_dict = json.loads(new_boundary.to_crs(4326).to_json())
with open("src/consts/boundary.ts", "w") as f:
    f.write("export const boundary = ")
    json.dump(geojson_dict, f)
    f.write(";")

In [2]:
def sample_line_points(line, spacing):
    """
    Returns evenly spaced points along a LineString.
    """
    length = line.length
    num_points = int(np.floor(length / spacing))
    distances = np.linspace(0, length, num_points, endpoint=False)
    return [line.interpolate(distance) for distance in distances]

In [3]:
spacing = 15
all_points = []
for geom in new_boundary.geometry:
    if geom.geom_type == 'LineString':
        all_points.extend(sample_line_points(geom, spacing))
    elif geom.geom_type == 'MultiLineString':
        for line in geom.geoms:
            all_points.extend(sample_line_points(line, spacing))
points_gdf = gpd.GeoDataFrame(geometry=all_points, crs=new_boundary.crs)

In [32]:
coords = points_gdf.to_crs(4326).geometry
latlon_list = [[pt.y, pt.x] for pt in coords]

# Write to TypeScript file
with open("src/consts/boundary-points.ts", "w") as f:
    f.write("export const boundaryPoints = ")
    json.dump(latlon_list, f)
    f.write(";")